In [7]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
import nltk
import spacy
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline 
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sophia.dellarciprete\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df2021= pd.read_csv('data/zeus_download_doc_digitalizzati20240214.csv', sep="|", encoding='latin')
df2021 = df2021.map(lambda x: x.lower() if isinstance(x, str) else x)
df2021.loc[df2021['famiglia'] == 'documenti non classificati', 'famiglia'] = 'altro'
df2021 = df2021[~df2021['nome_epc'].isna()]

df2021['original'] = df2021['nome_epc']
# Keep only alphabetic characters and spaces
df2021['nome_epc'] = df2021['original'].apply(lambda x: ''.join([c for c in x if c.isalpha() or c.isspace()]))
df2021['nome_epc'] = df2021['nome_epc'].str.replace(r'\s{2,}', ' ', regex=True)

# Trim leading and trailing whitespaces
df2021 = df2021.map(lambda x: x.strip() if isinstance(x, str) else x)

In [9]:
filtered_df = df2021[['famiglia', 'nome_epc', 'original']]
df_clean = filtered_df.drop_duplicates()

In [8]:
nlp = spacy.load('it_core_news_sm')
nltk.download('stopwords')

In [10]:
def procesar_texto(cadena):
    sinpuntuacion = [c for c in cadena if c not in string.punctuation]
    sinpuntuacion =' '.join(sinpuntuacion)
    palabras = sinpuntuacion.split()
    cadena_limpia = [palabra for palabra in palabras if palabra not in stopwords.words("italian")]
    return cadena_limpia

transformador = CountVectorizer(analyzer=procesar_texto).fit(df_clean['nome_epc'])

In [11]:
final_dataframe= df_clean[df_clean["famiglia"] == 'altro'] # To use once the model is working
df_clean = df_clean[df_clean["famiglia"] != 'altro']

In [14]:
x_train, x_test, y_train, y_test = train_test_split(df_clean["nome_epc"], df_clean["famiglia"],test_size=0.3)

pipeline = Pipeline([
('vectorizar',CountVectorizer(analyzer = procesar_texto)),
('transformar', TfidfTransformer()), 
('clasificar', RandomForestClassifier())
])

In [15]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectorizar',
                 CountVectorizer(analyzer=<function procesar_texto at 0x000001AD32959620>)),
                ('transformar', TfidfTransformer()),
                ('clasificar', RandomForestClassifier())])

In [16]:
predicciones = pipeline.predict(x_test)

In [17]:
print(classification_report(y_test, predicciones)) 

c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                              precision    recall  f1-score   support

                             atti giudiziali       0.57      0.58      0.57       773
                    certificazioni contabili       0.97      0.93      0.95      1114
                                     closing       0.00      0.00      0.00         5
                               comunicazioni       0.85      0.75      0.80      1219
                      comunicazioni provider       1.00      1.00      1.00     20525
         contratti e documentazione relativa       0.90      0.98      0.94      2154
                                    delibere       0.00      0.00      0.00         1
                                     fatture       0.99      1.00      0.99     24569
          garanzie e documentazione relativa       0.98      0.95      0.97      2791
                                     imposte       0.11      0.02      0.04        45
                         incidente_operativo       1.

c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
final_dataframe_copy = final_dataframe.copy()

In [19]:
predictions = pipeline.predict(final_dataframe['nome_epc'])

In [20]:
predictions_copy = predictions.copy()

In [21]:
final_dataframe['prev'] = predictions

In [22]:
final_dataframe = final_dataframe.drop(columns=['famiglia'])
final_dataframe.rename(columns={'prev': 'famiglia'}, inplace=True)
final_dataframe

,nome_epc,original,famiglia
2,inserimento acconto,inserimento acconto,comunicazioni
6,ordinanza di vendita ex art cpc senza incanto ...,ordinanza di vendita ex art. 571 c.p.c. (senza...,comunicazioni
11,il legale esterno ha esaminato il provvediment...,il legale esterno ha esaminato il provvediment...,garanzie e documentazione relativa
14,atto integrativo e di quietanza di finanziamen...,atto integrativo e di quietanza di finanziamen...,atti giudiziali
20,documento note sulla procedura,documento / note sulla procedura,visure
...,...,...,...
678802,legnotule,legnotule_488471_1_,fatture
678889,legnotule,legnotule_492664_1_,fatture
679118,nardo antoninopdf,nardo antonino_1.pdf,atti giudiziali
679163,il legale esterno allega istanza depositata di...,il legale esterno allega istanza depositata di...,atti giudiziali


In [23]:
result_dataframe = pd.concat([final_dataframe, df_clean], ignore_index=True)

In [24]:
df2021

,num_prat,famiglia,des_tipodoc,nomefile,nome_epc,url,original
0,1364164.0,visure,ipo-catastali,04425741.zip,propostedidelibera,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,proposte_di_delibera_451229__0
1,80949.0,comunicazioni provider,welcome letter - esitazione,03560445.pdf,,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,_173682_145_232927
2,213936.0,altro,altro,00290661.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto
3,939740.0,fatture,altro,04558436.zip,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_615795_1_
4,1019396.0,visure,ipo-catastali,03597376.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
...,...,...,...,...,...,...,...
679183,520972.0,perizie,valutazioni immobiliari,03723922.pdf,propostedidelibera,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,proposte_di_delibera_440827__0
679184,958229.0,certificazioni contabili,e/c,04399983.zip,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
679185,609923.0,fatture,altro,00834265.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_121546_1_
679186,777763.0,altro,altro,02256742.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento


In [25]:
prova = df2021.merge(result_dataframe, on='original', how='left')
prova

,num_prat,famiglia_x,des_tipodoc,nomefile,nome_epc_x,url,original,nome_epc_y,famiglia_y
0,1364164.0,visure,ipo-catastali,04425741.zip,propostedidelibera,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,proposte_di_delibera_451229__0,propostedidelibera,proposte
1,1364164.0,visure,ipo-catastali,04425741.zip,propostedidelibera,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,proposte_di_delibera_451229__0,propostedidelibera,visure
2,1364164.0,visure,ipo-catastali,04425741.zip,propostedidelibera,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,proposte_di_delibera_451229__0,propostedidelibera,proposte
3,1364164.0,visure,ipo-catastali,04425741.zip,propostedidelibera,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,proposte_di_delibera_451229__0,propostedidelibera,perizie
4,80949.0,comunicazioni provider,welcome letter - esitazione,03560445.pdf,,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,_173682_145_232927,,comunicazioni provider
...,...,...,...,...,...,...,...,...,...
4367002,1215155.0,visure,ipo-catastali,03002717.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,documentogenerico,pubblicita'
4367003,1215155.0,visure,ipo-catastali,03002717.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,documentogenerico,proposte
4367004,1215155.0,visure,ipo-catastali,03002717.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,documentogenerico,delibere
4367005,1215155.0,visure,ipo-catastali,03002717.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,documentogenerico,imposte


In [26]:
prova.famiglia_y.unique()

array(['proposte', 'visure', 'perizie', 'comunicazioni provider',
       'comunicazioni', 'certificazioni contabili', 'fatture', 'imposte',
       'atti giudiziali', 'contratti e documentazione relativa',
       "pubblicita'", 'garanzie e documentazione relativa',
       'passaggi di status e relativa documentazione', 'closing',
       'delibere', 'incidente_operativo'], dtype=object)

In [27]:
counts_dict = {}

# Iterate over unique 'famiglia' values
for famiglia_value in prova['famiglia_y'].unique():
    
    # Filter DataFrame for the current 'famiglia'
    filtered = prova[prova['famiglia_y'] == famiglia_value]
    
    # Count occurrences of each 'nome_epc' for the current 'famiglia'
    nome_epc_counts = filtered['nome_epc_y'].value_counts().to_dict()
    
    # Add counts to the dictionary
    counts_dict[famiglia_value] = nome_epc_counts

counts_df = pd.DataFrame(counts_dict)
counts_df

,proposte,visure,perizie,comunicazioni provider,comunicazioni,certificazioni contabili,fatture,imposte,atti giudiziali,contratti e documentazione relativa,pubblicita',garanzie e documentazione relativa,passaggi di status e relativa documentazione,closing,delibere,incidente_operativo
documentogenerico,121115.0,121115.0,121115.0,121115.0,242230.0,121115.0,121115.0,121115.0,121115.0,121115.0,121115.0,121115.0,121115.0,121115.0,121115.0,121115.0
documento note sulla procedura,39112.0,78224.0,39112.0,NaN,39112.0,39112.0,39112.0,39112.0,39112.0,39112.0,39112.0,39112.0,39112.0,39112.0,NaN,NaN
propostedidelibera,17230.0,3511.0,2748.0,NaN,811.0,120.0,26.0,NaN,971.0,99.0,NaN,99.0,15.0,27.0,NaN,NaN
il legale esterno ha esaminato comunicazione del professionista delegato di fissazione della vendita,7150.0,7150.0,7150.0,NaN,7150.0,NaN,NaN,NaN,14302.0,NaN,7152.0,7150.0,NaN,NaN,NaN,NaN
richiesta inserimento piano di rientro libero,4939.0,NaN,NaN,NaN,4939.0,9878.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
esame aggiornamento visura immobiliare fino alla data di iscrizione,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
informazioni ndg maffi davide lettera di messa in mora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
informazioni ndg piani mariangela ricevute di ritorno della lettera di messa in mora,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
incidenteoperativo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0


In [28]:
max_family_for_row = counts_df.idxmax(axis=1)

# Create a new DataFrame with 'word' as index and 'famiglia' as the corresponding column
result_df = pd.DataFrame({'word': max_family_for_row.index, 'famiglia': max_family_for_row.values})

result_df

,word,famiglia
0,documentogenerico,comunicazioni
1,documento note sulla procedura,visure
2,propostedidelibera,proposte
3,il legale esterno ha esaminato comunicazione d...,atti giudiziali
4,richiesta inserimento piano di rientro libero,certificazioni contabili
...,...,...
9657,esame aggiornamento visura immobiliare fino al...,passaggi di status e relativa documentazione
9658,informazioni ndg maffi davide lettera di messa...,passaggi di status e relativa documentazione
9659,informazioni ndg piani mariangela ricevute di ...,passaggi di status e relativa documentazione
9660,incidenteoperativo,incidente_operativo


In [29]:
done = df2021.merge(result_df, left_on='nome_epc', right_on='word', how='left')

In [30]:
done = done[done['famiglia_x']== 'altro']
done = done.drop(columns=['famiglia_x','word', 'nome_epc'] )
done.rename(columns={'famiglia_y': 'famiglia'}, inplace=True)
done.rename(columns={'original': 'nome_epc'}, inplace=True)
done

,num_prat,des_tipodoc,nomefile,url,nome_epc,famiglia
2,213936.0,altro,00290661.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,comunicazioni
6,39554.0,altro,01640843.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,ordinanza di vendita ex art. 571 c.p.c. (senza...,comunicazioni
11,1471376.0,altro,04457014.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno ha esaminato il provvediment...,garanzie e documentazione relativa
12,1018075.0,altro,01836366.tif,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,comunicazioni
14,580874.0,altro,01556180.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,atto integrativo e di quietanza di finanziamen...,atti giudiziali
...,...,...,...,...,...,...
678694,1203169.0,altro,04504194.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
678696,847112.0,altro,02268867.doc,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno allega istanza depositata di...,atti giudiziali
678701,662886.0,altro,02165909.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,esito udienza: il g.e. ha delegato professioni...,fatture
678705,1018178.0,altro,02280751.pdf,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,copia pdf altro formato,comunicazioni


In [31]:
done.to_csv('data/zeus_download_doc_digitalizzati20240214_exALTRO.csv', index=False)